# Undistort Dataset

In [1]:
class Args:
    uncalibrated_dataset_path = "../../resources/images/uncalibrated/calibration_dataset.npz"
    calibrated_dataset_path = "../../resources/images/calibrated/dataset_1.npz"
    calibration_matrix_path = "../../resources/calibrations/calibration.yml"
args = Args()

In [2]:
from models.dataset.DatasetHelper import openImageDataset, saveImageDataset, load_coefficients
from models.dataset.undistort import undistort_images

In [3]:
# Open Dataset and calibration file
rs_rgb, rs_z, zv_rgb, zv_z = openImageDataset(args.uncalibrated_dataset_path)
print(f"Opened dataset containing {len(rs_rgb)} image sets")

## Load Coefficients
transformation_matrix = load_coefficients(path=args.calibration_matrix_path)
print(f"Transformation matrix loaded:\n{transformation_matrix}")

## Undistort Images
zv_undistort_z, zv_undistort_rgb = undistort_images(zv_z, zv_rgb, transformation_matrix)

## save image dataset
saveImageDataset(
    args.calibrated_dataset_path,
    rs_rgb,
    rs_z,
    zv_undistort_rgb,
    zv_undistort_z
)

Opened dataset containing 24 image sets
Transformation matrix loaded:
[[ 6.28295449e-01 -1.55602841e-01 -2.04399827e-02  8.77084748e+01]
 [ 3.55724022e-02  9.00372850e-01 -1.68263636e-01 -9.98329375e+01]
 [ 1.60071381e-02  1.32399997e-01  5.86407508e-01  3.01591741e+01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
